In [54]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

In [55]:
# stocks data csv read
df = pd.read_csv('data.csv')
df = df.set_index('Date')

# s&p data csv read
df_sp = pd.read_csv('sp500.csv')
df_sp = df_sp.set_index('Date')

In [56]:
def date_slicer(df, start, duration, rebalancing_period=0):
    '''
    this function is used to slice out specific section of the data
    '''
    start = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=rebalancing_period))
    end = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=duration) - rd(days=1))
    return df.loc[start:end]

In [57]:
def data_process(df):
    '''
    this function gets the dataframe as input, processes it, and ouputs the cumulative change of the stocks
    that is used as input for training the model.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    df = df.to_numpy()
    df = torch.from_numpy(df).type(torch.Tensor)
    return df

In [58]:
def daily_change(df):
    '''
    this function calculate the daily change of stocks included in the dataframe.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    return df

In [59]:
def daily_return(df):
    '''
    this function calculate the daily return of stocks included in the dataframe, note that 
    daily return is equal to daily change + 1
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    return df

In [60]:
# shallow nnf biuld
class shallow_NNF(nn.Module):
    '''
    this class is used to train the data with Shallow NNF model, consisted of 2 fully connected layers, 
    a relU activation function in between and a softmax layer output that is translated into stock weights in portfolio.
    '''
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size) # fully connected layer
        self.fc2 = nn.Linear(hidden_size, num_classes) # fully connected layer
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [61]:
# deep nnf build
class deep_NNF(nn.Module):
    '''
    this class is used to train the data with Deep NNF model, consisted of 6 fully connected layers, 
    relU activation functions in between and a softmax layer output that is translated into stock weights in portfolio.
    dropout is also included in deep NNF model.
    '''
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1) # fully connected layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) # fully connected layer
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) # fully connected layer
        self.fc4 = nn.Linear(hidden_size3, hidden_size4) # fully connected layer
        self.fc5 = nn.Linear(hidden_size4, hidden_size5) # fully connected layer
        self.fc6 = nn.Linear(hidden_size5, num_classes) # fully connected layer
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [62]:
# 1/N model build
class equal_w_model():
    '''
    this class is used to construct a portfolio with equal weights.
    '''
    def __init__(self, df):
        self.df = df
        self.performance()
        
    def performance(self):
        self.df = np.array(self.df)
        weights = np.ones((len(self.df), 1)) * (1/len(self.df))
        cumulative_change = sum(np.multiply(weights, self.df.reshape(-1,1)))
        return cumulative_change, weights.reshape(-1)

In [63]:
# rebalancing period = one or three months
rbp = 1

# epochs
num_epochs = 100

In [64]:
# shallow_nnf hyperparameters
input_dim = 471
hidden_size = 471
num_classes = 471
lr = 1e-3 # learning rate

In [65]:
# shallow nnf tune
'''
loss function is set to MSE and Adam optimizer is used in this model.
'''
shallow_NNF = shallow_NNF(input_dim=input_dim, hidden_size=hidden_size, num_classes=num_classes)
shallow_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_optimizer = torch.optim.Adam(shallow_NNF.parameters(), lr=lr)

In [66]:
# deep_nnf hyperparameters
input_dim = 471
hidden_size1 = 471
hidden_size2 = 471
hidden_size3 = 471
hidden_size4 = 471
hidden_size5 = 471
num_classes = 471
lr = 1e-3 # learning rate
# probability of a neuron being shutdown that shuffles every epoch minimizing the overfit phenomenon
dropout_p = 0 

In [67]:
# deep nnf tune
'''
like in shallow NNF, loss function is set to MSE and Adam optimizer is used.
'''
deep_NNF = deep_NNF(input_dim=input_dim, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=num_classes)
deep_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_optimizer = torch.optim.Adam(deep_NNF.parameters(), lr=lr)

In [68]:
# RMSE
def RMSE(x, y, weights):
    '''
    this function calculates the root mean squere error of constructed portfollio and benchmark index 
    that is used for evaluating trained models.
    '''
    temp = 0
    for i in range(len(x)):
        temp += (sum(x.iloc[i] * weights) - y.iloc[i]) ** 2
    return math.sqrt(temp/len(x))

In [69]:
# MEAN
def MEAN(x, weights):
    '''
    this function calculates the mean return of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.mean()

In [70]:
# Volatility
def VOL(x, weights):
    '''
    this function calculates the volatility of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.std()

In [71]:
def portfolio_return(df, x_test, model, i, temp):   
    '''
    this function outputs the cumulative return of the portfolio test dataset of the given dataframe
    ''' 
    x_return = date_slicer(df, '2018-01-01', 1, i)
    x_return =  x_return.pct_change()
    x_return =  x_return.tail(-1)
    x_return =  x_return + 1
    x_return =  x_return.cumprod()
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    for i in range(len(x_return)):
        temp.append(sum(x_return.iloc[i] * weights))
    temp = np.array(temp)
    return temp

In [72]:
def index_return(df_sp, i, temp):
    '''
    this function outputs the cumulative return of the benchmark index test dataset of the given dataframe
    '''
    y_return = date_slicer(df_sp, '2018-01-01', 1, i)
    y_return = y_return.pct_change()
    y_return = y_return.tail(-1)
    y_return = y_return + 1
    y_return = y_return.cumprod()
    
    for i in range(len(y_return)):
        temp.append(sum(y_return.iloc[i]))
    temp = np.array(temp)
    return temp

In [73]:
def valid_fun(x_valid, i, model):
    '''
    this function gets validation dataset, model and rebalaning period as input, then outputs the RMSE of given dataset.
    '''
    x_change = daily_change(date_slicer(df, '2017-07-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2017-07-01', 6, i))
    # x_return = daily_return(date_slicer(df, '2017-07-01', 6, i))
    # y_return = daily_return(date_slicer(df_sp, '2017-07-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_valid).performance()[1]
    else:
        weights = np.array(model(x_valid)[1].detach())
    
    valid_rmse = RMSE(x_change, y_change, weights)
    # valid_mean = MEAN(x_return, weights)
    # valid_vol  = VOL(x_return, weights)
    
    print(f'Validation RMSE: {valid_rmse}')
    # print(f'Validation MEAN: {valid_mean}')
    # print(f'Validation VOL: {valid_vol}')
    
    return valid_rmse

In [74]:
def test_fun(x_test, i, model):
    '''
    this function gets test dataset, model and rebalaning period as input, then outputs the RMSE, Mean and volatility 
    of the given dataset.
    '''
    x_change = daily_change(date_slicer(df, '2018-01-01', 1, i))
    y_change = daily_change(date_slicer(df_sp, '2018-01-01', 1, i))
    x_return = daily_return(date_slicer(df, '2018-01-01', 1, i))
    y_return = daily_return(date_slicer(df_sp, '2018-01-01', 1, i))
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    test_rmse = RMSE(x_change, y_change, weights)
    test_mean = MEAN(x_return, weights)
    test_vol  = VOL(x_return, weights)
    test_dic = {'RMSE': test_rmse, 'MEAN': test_mean, 'VOL': test_vol} # a dictionary for storing the results
    
    print(f'Test RMSE: {test_rmse}')
    print(f'Test MEAN: {test_mean}')
    print(f'Test VOL: {test_vol}')
    
    return test_dic

### **Deep NNF Training**

In [75]:
# deep nnf training function
'''
this function is used to train the model using x_train & y_train given to it, printing MSE of trained model in first and last
 epich and also printing train time of the model
'''
def train_deep_nnf(x_train, y_train, i):
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF(x_train)[0]
        loss_deep_nnf = deep_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [76]:
# deep nnf
'''
in this cell,firstly, train, validation and test datasets are sliced in each loop. then deep NNf model gets trained and
then best model will be chosen. Also RMSE, Mean and volatility of all models and then the best model is printed.
'''
deep_nnf_valid_rmse_list = []
deep_nnf_test_results = []
deep_nnf_test_plot = [] # storing the deep model test data return for plotting later on
index_test_plot = [] # storing the index test data return for plotting later on

for i in range(int(24/rbp)):
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    train_deep_nnf(x_train, y_train, i*rbp)
    deep_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, deep_NNF))
    deep_nnf_test_results.append(test_fun(x_test, i*rbp, deep_NNF))
    portfolio_return(df, x_test, deep_NNF, i, deep_nnf_test_plot)
    index_return(df_sp, i, index_test_plot)
    deep_NNF.reset_parameters()

print(f'\nMin Valid RMSE is: {min(deep_nnf_valid_rmse_list)} for model i = {deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))+1}')
print('Selected Model Test Results are:')
print('RMSE =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['RMSE'])
print('MEAN =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['MEAN'])
print('VOL =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['VOL'])

deep_best_result_index = deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))
deep_nnf_test_plot = np.array(deep_nnf_test_plot).reshape(-1,1)
index_test_plot = np.array(index_test_plot).reshape(-1,1)


Deep NNF Training & Results for model 1.0:
Epoch 1 of 10 | MSE: 0.04568424075841904
Epoch 10 of 10 | MSE: 0.0004861154593527317
Training time: 0.19
Validation RMSE: 0.0014704474061788399


/Users/hamidrezarahimzadeh/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Test RMSE: 0.0014339130477000297
Test MEAN: 1.001897959974057
Test VOL: 0.005360620568554472

Deep NNF Training & Results for model 2.0:
Epoch 1 of 10 | MSE: 0.05325326323509216
Epoch 10 of 10 | MSE: 0.0009558473248034716
Training time: 0.09
Validation RMSE: 0.001614403839352181
Test RMSE: 0.0022678555532461286
Test MEAN: 0.9979034756115183
Test VOL: 0.015437859250082588

Deep NNF Training & Results for model 3.0:
Epoch 1 of 10 | MSE: 0.04559122398495674
Epoch 10 of 10 | MSE: 0.012366250157356262
Training time: 0.10
Validation RMSE: 0.001753891656552477
Test RMSE: 0.002210022824702207
Test MEAN: 1.0002369551915147
Test VOL: 0.010991512476734753

Deep NNF Training & Results for model 4.0:
Epoch 1 of 10 | MSE: 0.05981092154979706
Epoch 10 of 10 | MSE: 0.0026019408833235502
Training time: 0.13
Validation RMSE: 0.0019469596327262688
Test RMSE: 0.0017350709258675348
Test MEAN: 1.001311298252053
Test VOL: 0.008721380514640905

Deep NNF Training & Results for model 5.0:
Epoch 1 of 10 | MSE: 0

### **Shallow NNF Training**

In [77]:
# shallow nnf training function
'''
this function is used to train the model using x_train & y_train given to it, printing MSE of trained model in first and last
epoch and also printing train time of the model
'''
def train_shallow_nnf(x_train, y_train, i):
    start_time_shallow_nnf = time.time()
    print(f'\nShallow NNF Training & Results for model {(i/rbp)+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF(x_train)[0]
        loss_shallow_nnf = shallow_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [78]:
#shallow nnf
'''
in this cell,firstly, train, validation and test datasets are sliced in each loop. then shallow NNf model gets trained and
then best model will be chosen. Also RMSE, Mean and volatility of all models and then the best model is printed.
'''
shallow_nnf_valid_rmse_list = []
shallow_nnf_test_results = [] 
shallow_nnf_test_plot = [] # storing the shallow model test data return for plotting later on

for i in range(int(24/rbp)):
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    
    train_shallow_nnf(x_train, y_train, i*rbp)
    shallow_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, shallow_NNF))
    shallow_nnf_test_results.append(test_fun(x_test, i*rbp, shallow_NNF))
    portfolio_return(df, x_test, shallow_NNF, i, shallow_nnf_test_plot)
    shallow_NNF.reset_parameters()
    

# print(f'\nMin Valid RMSE is: {min(valid_rmse_list)} for model i = {(deep_best_result_index)+1}')
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', shallow_nnf_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', shallow_nnf_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', shallow_nnf_test_results[(deep_best_result_index)]['VOL'])

shallow_nnf_test_plot = np.array(shallow_nnf_test_plot).reshape(-1,1)


Shallow NNF Training & Results for model 1.0:
Epoch 1 of 10 | MSE: 0.0436849407851696
Epoch 10 of 10 | MSE: 0.00036164841731078923
Training time: 0.10
Validation RMSE: 0.0014737933842850386
Test RMSE: 0.0014335311847101513
Test MEAN: 1.0019152986298194
Test VOL: 0.005343951406957578

Shallow NNF Training & Results for model 2.0:
Epoch 1 of 10 | MSE: 0.05439304560422897
Epoch 10 of 10 | MSE: 0.007282321806997061
Training time: 0.09
Validation RMSE: 0.001547898324354915
Test RMSE: 0.0022317113839817955
Test MEAN: 0.9979281083421905
Test VOL: 0.015463557649808425

Shallow NNF Training & Results for model 3.0:
Epoch 1 of 10 | MSE: 0.04768081381917
Epoch 10 of 10 | MSE: 0.006824583746492863
Training time: 0.09
Validation RMSE: 0.0016995933456803746
Test RMSE: 0.002137346894756386
Test MEAN: 1.0002453144617798
Test VOL: 0.011020957820500847

Shallow NNF Training & Results for model 4.0:
Epoch 1 of 10 | MSE: 0.05400165542960167
Epoch 10 of 10 | MSE: 0.005171079188585281
Training time: 0.07
V

### **1/N Model**

In [79]:
'''
here we run the 1/N model, for the number of stocks, each stock gets the weight of 1/N meaning that
every stock is equally important, this model play the role of a benchmark to see how effective our model are
'''
equal_w_model_valid_rmse_list = []
equal_w_model_test_results = []
equal_w_model_test_plot = [] # storing the 1/n model test data return for plotting later on

for i in range(int(24/rbp)):
    print(f'\nEqual Weights Model Results for model {i+1}:')
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    
    equal_w_model_valid_rmse_list.append(valid_fun(x_valid, i*rbp, equal_w_model))
    equal_w_model_test_results.append(test_fun(x_test, i*rbp, equal_w_model))
    portfolio_return(df, x_test, equal_w_model, i, equal_w_model_test_plot)
    
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', equal_w_model_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', equal_w_model_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', equal_w_model_test_results[(deep_best_result_index)]['VOL'])

equal_w_model_test_plot = np.array(equal_w_model_test_plot).reshape(-1,1)


Equal Weights Model Results for model 1:
Validation RMSE: 0.0013752466607634818
Test RMSE: 0.00141308068973907
Test MEAN: 1.00194947794337
Test VOL: 0.005348201810361211

Equal Weights Model Results for model 2:
Validation RMSE: 0.0014103582761839522
Test RMSE: 0.002225473810475016
Test MEAN: 0.9979382024584036
Test VOL: 0.015471723202182033

Equal Weights Model Results for model 3:
Validation RMSE: 0.00161101492926301
Test RMSE: 0.0021228799964131116
Test MEAN: 1.0002504247997188
Test VOL: 0.011031703269168955

Equal Weights Model Results for model 4:
Validation RMSE: 0.00178815568268382
Test RMSE: 0.0016167972928903618
Test MEAN: 1.001305977361471
Test VOL: 0.008808539875808678

Equal Weights Model Results for model 5:
Validation RMSE: 0.0017533218637125797
Test RMSE: 0.0012496906685729259
Test MEAN: 1.0007751300633236
Test VOL: 0.0062942423298088844

Equal Weights Model Results for model 6:
Validation RMSE: 0.0017002638599400465
Test RMSE: 0.0012049049143032751
Test MEAN: 1.0001325

In [80]:
# print test results
'''
here we compare the results in a dataframe featuring RMSE, MEAN and, volatility of each model in the test dataset
that has the best results for deep nnf model. this dataframe can cope with the understanding of why we bother 
implementing a complex neural network
'''
print(f'Models test results with rebalancing period of {rbp} month(s) are: ')
deep_temp = pd.DataFrame(deep_nnf_test_results)
deep_temp = deep_temp.iloc[deep_best_result_index]
shallow_temp = pd.DataFrame(shallow_nnf_test_results)
shallow_temp = shallow_temp.iloc[deep_best_result_index]
equal_w_temp = pd.DataFrame(equal_w_model_test_results)
equal_w_temp = equal_w_temp.iloc[deep_best_result_index]

# extract the mean and volatility of the s&p index on the test dataset
sp_temp_rmse = '-'
sp_temp_mean = daily_return(date_slicer(df_sp, '2018-01-01', 1, deep_best_result_index)).mean()[0]
sp_temp_std = daily_return(date_slicer(df_sp, '2018-01-01', 1, deep_best_result_index)).std()[0]
sp_temp = pd.DataFrame([sp_temp_rmse, sp_temp_mean, sp_temp_std], index=deep_temp.index)

# concatinating the result in a unified dataframe
final_result = pd.concat([deep_temp, shallow_temp, equal_w_temp, sp_temp], axis=1, join='inner')
final_result.columns = ['Deep NNF', 'Shallow NNF', '1/N Model', 'S&P 500']
final_result

Models test results with rebalancing period of 1 month(s) are: 


,Deep NNF,Shallow NNF,1/N Model,S&P 500
RMSE,0.001434,0.001434,0.001413,-
MEAN,1.001898,1.001915,1.001949,1.002337
VOL,0.005361,0.005344,0.005348,0.005601


In [81]:
'''
to further showcase the results, here we compute the average RMSE of each model in test dataset
'''
print(f'Average of test RMSE for each model: ')

deep_nnf_test_rmse_mean = 0 # temp variable for storing each tmse for deep nnf model
for i in range(int(24/rbp)):
    deep_nnf_test_rmse_mean += deep_nnf_test_results[i]['RMSE']
print(f'Deep NNF: {deep_nnf_test_rmse_mean/int(24/rbp)}')

shallow_nnf_test_rmse_mean = 0 # temp variable for storing each tmse for shallow nnf model
for i in range(int(24/rbp)):
    shallow_nnf_test_rmse_mean += shallow_nnf_test_results[i]['RMSE']
print(f'Shallow NNF: {shallow_nnf_test_rmse_mean/int(24/rbp)}')

equal_w_model_test_rmse_mean = 0 # temp variable for storing each tmse for 1/n model model
for i in range(int(24/rbp)):
    equal_w_model_test_rmse_mean += equal_w_model_test_results[i]['RMSE']
print(f'Equal weight model: {equal_w_model_test_rmse_mean/int(24/rbp)}')

Average of test RMSE for each model: 
Deep NNF: 0.0018668248427246078
Shallow NNF: 0.0018336460979335976
Equal weight model: 0.001811103162995504


In [82]:
# concatinating the test dataset return results of each model + index return for plot
plot_test = pd.concat([pd.DataFrame(deep_nnf_test_plot), pd.DataFrame(shallow_nnf_test_plot),
                       pd.DataFrame(equal_w_model_test_plot), pd.DataFrame(index_test_plot)], axis=1, join='inner')
plot_test.columns = ['Deep NNF', 'Shallow NNF', '1/N Model', 'S&P 500']

In [83]:
# importing a module for better and more interactive plot
import cufflinks as cf
cf.set_config_file(offline = True)

'''
plotting deep nnf, shallow nnf and, 1/n model performance on the test dataset, compare them with
index (s&p) for better understanding
'''
plot_test.iplot()